In [1]:
from skilift import GTFSFeed, get_stop_vertex, read_osm, get_elevations_for_nodes, \
get_vertex_nodes, StreetData, OnEarthSurfaceNode, TransitEdgeProvider, StreetEdgeProvider

In [2]:
import pandas as pd
from zipfile import ZipFile
from collections import defaultdict
from typing import Dict, Set, Tuple, Iterable, Iterator, Any
import numpy as np
from shapely.geometry import Point, LineString, MultiLineString
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
fn = "/home/bmander/skilift_data/transit/redding.zip"

feed = GTFSFeed(fn)
transit_edge_provider = TransitEdgeProvider(feed)

In [4]:
# get epoch time on Aptil 19, 2023 at 1:00pm pacific time
t0 = pd.Timestamp("2023-04-19 13:00:00", tz="America/Los_Angeles")
t0

Timestamp('2023-04-19 13:00:00-0700', tz='America/Los_Angeles')

In [5]:
node0 = get_stop_vertex(feed, "Downtown Transit Center", t0)
node0

AtStopvertex(stop_id:2000, datetime:2023-04-19 13:00:00-07:00)

In [6]:
node1 = transit_edge_provider.outgoing(node0)[0].vertex
node1

DepartureVertex(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:0, datetime:2023-04-19 14:25:00-07:00)

In [7]:
node2 = transit_edge_provider.outgoing(node0)[0].vertex
node2

DepartureVertex(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:0, datetime:2023-04-19 14:25:00-07:00)

In [8]:
transit_edge_provider.outgoing(node2)

[Edge(vertex=ArrivalVertex(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:1, datetime:2023-04-19 14:35:00-07:00), weight=600.0)]

In [9]:
osm_filename = "/home/bmander/skilift_data/street/redding.pbf"


In [10]:
elev_geotiff_filename = "/home/bmander/skilift_data/elevation/USGS_13_n41w123_20210624.tif"

In [11]:
elev_dataset = StreetData(osm_filename, elev_geotiff_filename)

Reading OSM file...done
Indexing ways...done
Getting node elevations...done
Creating spatial index...done


In [12]:
nd_points = elev_dataset.nodes.values()
lon = sum([x[0] for x in nd_points])/len(nd_points)
lat = sum([x[1] for x in nd_points])/len(nd_points)
lon, lat

(-122.35873694849425, 40.584176490162584)

In [13]:
node0 = OnEarthSurfaceNode(lon, lat, t0)
node0

OnEarthSurfaceNode(-122.35873694849425, 40.584176490162584, 2023-04-19 13:00:00-07:00)

In [14]:
street_edge_provider = StreetEdgeProvider(elev_dataset)
v0 = street_edge_provider.outgoing(node0)[0].vertex
v0

MidStreetNode(774909443, 1, 0.573216480698101, 2023-04-19 13:00:50.220285374-07:00)

In [15]:
way = street_edge_provider.osm_data.ways[774909443]

In [16]:
street_edge_provider.osm_data.node_refs[way.nds[5]]

{(675408601, 0), (774909443, 5)}

In [17]:
street_edge_provider.osm_data.next_vertex_node(v0.way_id, 1, search_forward=False)

0

In [18]:
seg_start = 1
seg_end = 0

In [19]:
way.nds

[5797734026, 5797733850, 5797733851, 5797733852, 5797733953, 5797734025]

In [20]:
way.nds[seg_start: None: -1]

[5797733850, 5797734026]

In [21]:
street_edge_provider.outgoing(v0)

[Edge(vertex=EndStreetNode(774909443, 5, 2023-04-19 13:01:59.235587877-07:00), weight=69.0153025031656),
 Edge(vertex=EndStreetNode(774909443, 0, 2023-04-19 13:01:46.456623367-07:00), weight=56.236337993785625)]